In [ ]:
#Hedging our bets muni-style

In [ ]:
#import statements for packages/libraries
#which libraries do we need?
#pyfolio
#scipy
#backtester

In [ ]:
#pull in data from API (7-ish years of data 1/1/2013-11/01/2020)
#bonds--bloomberg
#muni etf (MUB)--alpaca
#swap--bloomberg or alpaca
#treasury--quandl
#treasury ETF (TLT)--alpaca

In [ ]:
# use pandas to create dataframers

In [ ]:
# clean data (drop nulls/duplicates)

In [ ]:
# convert price/yield/rate data into daily change in value

In [ ]:
#function passing a cusip to put it against hedges
#what are the steps we need our function to do:
    #1.pick bond
    #2. compare versus all hedges
    #3. show correlation of bond vis a vis different hedges

In [ ]:
##concat separate dataframes (bonds, muni etf, swap, treasury, treasury etf)

In [ ]:
## calculate standard deviation

In [ ]:
## calculate sharpe ratios

In [ ]:
# correlation matrix for individual bond vs different hedges--visualization with interaction?

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline

In [61]:
ETF_csv = Path("Resources/ETFs.csv")
ETF_df = pd.read_csv(ETF_csv, infer_datetime_format=True, parse_dates=True)
# ETF_df.sort_index(inplace=True)
ETF_df.rename(columns={'Unnamed: 0': 'Date'}, inplace = True)
ETF_df.head()

,Date,JNK,JNK.1,JNK.2,JNK.3,JNK.4,LQD,LQD.1,LQD.2,LQD.3,...,UDN,UDN.1,UDN.2,UDN.3,UDN.4,UUP,UUP.1,UUP.2,UUP.3,UUP.4
0,NaN,open,high,low,close,volume,open,high,low,close,...,open,high,low,close,volume,open,high,low,close,volume
1,2013-01-02 00:00:00-05:00,41.05,41.05,40.96,41.03,6801481,120.96,121.31,120.81,121.27,...,27.28,27.3099,27.15,27.18,33539.0,21.74,21.85,21.72,21.82,782706.0
2,2013-01-03 00:00:00-05:00,40.95,41.03,40.89,40.94,6337005,121.14,121.18,120.56,120.6,...,27.08,27.1,26.95,26.96,72918.0,21.9,22.0,21.8873,21.99,1091846.0
3,2013-01-04 00:00:00-05:00,40.99,41.05,40.88,40.99,5081128,120.57,120.649,120.26,120.55,...,26.9,26.97,26.89,26.95,146499.0,22.03,22.05,21.98,21.99,1022712.0
4,2013-01-07 00:00:00-05:00,40.93,41.09,40.89,41.08,5909999,120.69,120.8,120.51,120.72,...,26.94,27.04,26.932,27.04,41209.0,21.9901,22.01,21.92,21.93,1910335.0


,JNK,JNK.1,JNK.2,JNK.3,JNK.4,LQD,LQD.1,LQD.2,LQD.3,LQD.4,...,UDN,UDN.1,UDN.2,UDN.3,UDN.4,UUP,UUP.1,UUP.2,UUP.3,UUP.4
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02 00:00:00-05:00,41.05,41.05,40.96,41.03,6801481,120.96,121.31,120.81,121.27,3392382,...,27.28,27.3099,27.15,27.18,33539.0,21.74,21.85,21.72,21.82,782706.0
2013-01-03 00:00:00-05:00,40.95,41.03,40.89,40.94,6337005,121.14,121.18,120.56,120.6,2871556,...,27.08,27.1,26.95,26.96,72918.0,21.9,22.0,21.8873,21.99,1091846.0
2013-01-04 00:00:00-05:00,40.99,41.05,40.88,40.99,5081128,120.57,120.649,120.26,120.55,3998513,...,26.9,26.97,26.89,26.95,146499.0,22.03,22.05,21.98,21.99,1022712.0
2013-01-07 00:00:00-05:00,40.93,41.09,40.89,41.08,5909999,120.69,120.8,120.51,120.72,2471978,...,26.94,27.04,26.932,27.04,41209.0,21.9901,22.01,21.92,21.93,1910335.0
2013-01-08 00:00:00-05:00,41.08,41.1,41.02,41.06,2958198,120.81,120.94,120.76,120.85,2138549,...,26.99,27.01,26.961,27.005,38866.0,21.96,21.99,21.95,21.95,1134138.0


In [65]:
ETF_df['Date'] = pd.to_datetime(ETF_df['Date'], utc=True)
# print(ETF_df.dtypes)
ETF_df['Date']= ETF_df['Date'].dt.strftime('%m/%d/%Y')
ETF_df.head()

,Date,JNK,JNK.1,JNK.2,JNK.3,JNK.4,LQD,LQD.1,LQD.2,LQD.3,...,UDN,UDN.1,UDN.2,UDN.3,UDN.4,UUP,UUP.1,UUP.2,UUP.3,UUP.4
0,NaN,open,high,low,close,volume,open,high,low,close,...,open,high,low,close,volume,open,high,low,close,volume
1,01/02/2013,41.05,41.05,40.96,41.03,6801481,120.96,121.31,120.81,121.27,...,27.28,27.3099,27.15,27.18,33539.0,21.74,21.85,21.72,21.82,782706.0
2,01/03/2013,40.95,41.03,40.89,40.94,6337005,121.14,121.18,120.56,120.6,...,27.08,27.1,26.95,26.96,72918.0,21.9,22.0,21.8873,21.99,1091846.0
3,01/04/2013,40.99,41.05,40.88,40.99,5081128,120.57,120.649,120.26,120.55,...,26.9,26.97,26.89,26.95,146499.0,22.03,22.05,21.98,21.99,1022712.0
4,01/07/2013,40.93,41.09,40.89,41.08,5909999,120.69,120.8,120.51,120.72,...,26.94,27.04,26.932,27.04,41209.0,21.9901,22.01,21.92,21.93,1910335.0


,Date,JNK,JNK.1,JNK.2,JNK.3,JNK.4,LQD,LQD.1,LQD.2,LQD.3,...,UDN,UDN.1,UDN.2,UDN.3,UDN.4,UUP,UUP.1,UUP.2,UUP.3,UUP.4
0,NaN,open,high,low,close,volume,open,high,low,close,...,open,high,low,close,volume,open,high,low,close,volume
1,01/02/2013,41.05,41.05,40.96,41.03,6801481,120.96,121.31,120.81,121.27,...,27.28,27.3099,27.15,27.18,33539.0,21.74,21.85,21.72,21.82,782706.0
2,01/03/2013,40.95,41.03,40.89,40.94,6337005,121.14,121.18,120.56,120.6,...,27.08,27.1,26.95,26.96,72918.0,21.9,22.0,21.8873,21.99,1091846.0
3,01/04/2013,40.99,41.05,40.88,40.99,5081128,120.57,120.649,120.26,120.55,...,26.9,26.97,26.89,26.95,146499.0,22.03,22.05,21.98,21.99,1022712.0
4,01/07/2013,40.93,41.09,40.89,41.08,5909999,120.69,120.8,120.51,120.72,...,26.94,27.04,26.932,27.04,41209.0,21.9901,22.01,21.92,21.93,1910335.0


In [66]:
ETF_df = ETF_df.drop(ETF_df.index[0]).set_index('Date')
ETF_df.head()

,JNK,JNK.1,JNK.2,JNK.3,JNK.4,LQD,LQD.1,LQD.2,LQD.3,LQD.4,...,UDN,UDN.1,UDN.2,UDN.3,UDN.4,UUP,UUP.1,UUP.2,UUP.3,UUP.4
Date,,,,,,,,,,,,,,,,,,,,,
01/02/2013,41.05,41.05,40.96,41.03,6801481,120.96,121.31,120.81,121.27,3392382,...,27.28,27.3099,27.15,27.18,33539.0,21.74,21.85,21.72,21.82,782706.0
01/03/2013,40.95,41.03,40.89,40.94,6337005,121.14,121.18,120.56,120.6,2871556,...,27.08,27.1,26.95,26.96,72918.0,21.9,22.0,21.8873,21.99,1091846.0
01/04/2013,40.99,41.05,40.88,40.99,5081128,120.57,120.649,120.26,120.55,3998513,...,26.9,26.97,26.89,26.95,146499.0,22.03,22.05,21.98,21.99,1022712.0
01/07/2013,40.93,41.09,40.89,41.08,5909999,120.69,120.8,120.51,120.72,2471978,...,26.94,27.04,26.932,27.04,41209.0,21.9901,22.01,21.92,21.93,1910335.0
01/08/2013,41.08,41.1,41.02,41.06,2958198,120.81,120.94,120.76,120.85,2138549,...,26.99,27.01,26.961,27.005,38866.0,21.96,21.99,21.95,21.95,1134138.0


In [70]:
ETF_df['JNK.3'] = ETF_df['JNK.3'].astype(float)
print(ETF_df.dtypes)

JNK      float64
JNK.1     object
JNK.2     object
JNK.3    float64
JNK.4     object
LQD       object
LQD.1     object
LQD.2     object
LQD.3     object
LQD.4     object
MUB       object
MUB.1     object
MUB.2     object
MUB.3     object
MUB.4     object
TLT       object
TLT.1     object
TLT.2     object
TLT.3     object
TLT.4     object
UDN       object
UDN.1     object
UDN.2     object
UDN.3     object
UDN.4     object
UUP       object
UUP.1     object
UUP.2     object
UUP.3     object
UUP.4     object
dtype: object


In [ ]:
df.rename(columns={'JNK.3': 'JNK',
                   'LQD.3': 'LQD',
                   'MUB.3': 'MUB',
                   'TLT.3': 'TLT',
                   'UDN.3': 'UDN',
                   'UUP.3': 'UUP',
                  }, inplace=True)